In [1]:
import pandas as pd
import numpy as np

In [5]:
def load_all_data(file_path):
    data = pd.read_json(file_path, orient='records', lines=True)
    data = data.iloc[0:765,:4]
    return data

In [6]:
def tmp_label_process(labeled_data):
    high_low_labels = labeled_data['quality'].apply(lambda x:1 if x == 1 else 0)
    return high_low_labels

In [7]:
file_path = 'D:\Research\RQ1\modelcard_data (update).json'

labeled_data = load_all_data(file_path)

labeled_data

,modelId,modelcard_text,modelcard_titles_combined,quality
0,DrishtiSharma/wav2vec2-large-xls-r-300m-ab-v4,\n<!-- This model card has been generated auto...,Model description Intended uses & limitations...,0
1,Helsinki-NLP/opus-mt-es-nl,\n### opus-mt-es-nl\n\n* source languages: es\...,opus-mt-es-nl Benchmarks,0
2,Helsinki-NLP/opus-mt-sv-tiv,\n### opus-mt-sv-tiv\n\n* source languages: sv...,opus-mt-sv-tiv Benchmarks,0
3,MultiBertGunjanPatrick/multiberts-seed-15,# MultiBERTs Seed 15 (uncased)\n\nSeed 15 Mult...,MultiBERTs Seed 15 (uncased) Model description...,0
4,SauravMaheshkar/clr-finetuned-bert-large-uncased,![](https://github.com/SauravMaheshkar/CommonL...,FineTuning,0
...,...,...,...,...
760,Niggendar/waiANINSFWPONYXL_v20,\n# Model Card for Model ID\n\n<!-- Provide a ...,Model Card for Model ID Model Details Model De...,0
761,jayasuryajsk/Llama-3-8b-Telugu-Romanized,# Llama-3-8b-Telugu_Romanized\n\nLlama-3 8B fi...,Llama-3-8b-Telugu_Romanized Model Details Mode...,0
762,mradermacher/Chaotic-Soliloquy-4x8B-GGUF,## About\n\n<!-- ### quantize_version: 1 -->\n...,About Usage Provided Quants FAQ / Model Reques...,0
763,gradientai/Llama-3-8B-Instruct-262k,"<img src=""https://cdn-uploads.huggingface.co/p...",Llama-3 8B Instruct 262k The Gradient AI Team ...,1


In [10]:
labeled_data.iloc[0 , 2]

' Model description Intended uses & limitations Training and evaluation data Training procedure Training hyperparameters Training results Framework versions'

In [5]:
good_bad_labels = tmp_label_process(labeled_data)

In [6]:
pos_num = np.sum(good_bad_labels == 1)

In [7]:
print("Positive samples:", pos_num)

Positive samples: 54


In [ ]:
def main(file_path, model_type):
    mylogger = logger.mylog("dl")
    labeled_data = load_all_data(file_path)
    good_bad_labels = tmp_label_process(labeled_data)
    pos_num = np.sum(good_bad_labels == 1)
    print("Positive samples:", pos_num)
    glo._init()
    fold_num = 0
    kf = KFold(n_splits=10, random_state=3407, shuffle=True)
    for train_index, test_index in kf.split(labeled_data):
        train_x, train_y = list(labeled_data.iloc[train_index, 2]), list(good_bad_labels.iloc[train_index])
        test_x, test_y = list(labeled_data.iloc[test_index, 2]), list(good_bad_labels.iloc[test_index])
        # Use single input models (CNN, bi-lstm, transformer)
        dl_container(train_x, train_y, test_x, test_y, mylogger, "score", fold_num, model_type)
        fold_num += 1

    ## calculate 10-fold metrics at last
    mylogger.info("==================Todo comment good_bad classifers=============")
    cal_last_metrics(mylogger, model_type, "score")